In [1]:
import pathlib, os, json
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['HF_HOME'] = str(pathlib.Path("~/scratch-llm/storage/cache/huggingface/").expanduser().absolute()) # '/scratch-llm/storage/cache/'
# os.environ["TRANSFORMERS_CACHE"] = "~/scratch-llm/storage/models/"

import torch, pickle
import numpy as np

from transformers import AutoTokenizer
from nebulagraph_lite import nebulagraph_let as ng_let
from llama_index.graph_stores.nebula import NebulaPropertyGraphStore

from llama_index.core import Settings
from llama_index.core.schema import TextNode
from llama_index.core.prompts import PromptTemplate
from llama_index.core.base.llms.types import ChatMessage
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.vector_stores.simple import SimpleVectorStoreData, SimpleVectorStore, VectorStoreQuery

from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from typing import List
from numpy import dot
from numpy.linalg import norm
from pydantic import BaseModel, Field
from llama_index.core.output_parsers import PydanticOutputParser

# NebulaGraph conexion

In [2]:
# load NebulaGraph JupyterNotebook extension
# !udocker pull vesoft/nebula-metad:v3
# !udocker create --name=nebula-metad vesoft/nebula-metad:v3
# !udocker setup --execmode=F1 nebula-metad
# !udocker pull vesoft/nebula-graphd:v3
# !udocker create --name=nebula-graphd vesoft/nebula-graphd:v3
# !udocker setup --execmode=F1 nebula-graphd
# !udocker pull vesoft/nebula-storaged:v3
# !udocker create --name=nebula-storaged vesoft/nebula-storaged:v3
# !udocker setup --execmode=F1 nebula-storaged

n = ng_let(in_container=True)
n.start() # Takes around 5 mins

Message: Activating storaged...
Result of `SHOW HOSTS`:
    errors:
        code: 0
    results:
        spaceName: 
        data:
            meta:
                None, None, None, None, None, None, None
            row:
                127.0.0.1, 9779, ONLINE, 201, PrimeKG:100, PrimeKG_nebula:100, basketballplayer:1, PrimeKG:100, PrimeKG_nebula:100, basketballplayer:1, 3.8.0
        columns:
            Host, Port, Status, Leader count, Leader distribution, Partition distribution, Version
        errors:
            code: 0
        latencyInUs: 1464
Info: loading basketballplayer dataset...

  _   _      _           _        ____                 _     
 | \ | | ___| |__  _   _| | __ _ / ___|_ __ __ _ _ __ | |__  
 |  \| |/ _ | '_ \| | | | |/ _` | |  _| '__/ _` | '_ \| '_ \ 
 | |\  |  __| |_) | |_| | | (_| | |_| | | | (_| | |_) | | | |
 |_| \_|\___|_.__/ \__,_|_|\__,_|\____|_|  \__,_| .__/|_| |_|
                                                |_|          
                          

In [3]:
%reload_ext ngql
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

[OK] Connection Pool Created


,Name
0,PrimeKG
1,PrimeKG_nebula
2,basketballplayer


# Vector + Graph store

## SimpleVectorStore:

In [4]:
# Load the actual data into all_nodes_embeddded
with open(os.path.expanduser('~/scratch-llm/storage/nodes/all_nodes_all-mpnet-base-v2.pkl'), 'rb') as f:
    all_nodes_embedded: List[TextNode] = pickle.load(f)
# Create dictionaries from the nodes
embedding_dict = {node.id_: node.get_embedding() for node in all_nodes_embedded}
text_id_to_ref_doc_id = {node.id_: node.ref_doc_id or "None" for node in all_nodes_embedded}
metadata_dict = {node.id_: node.metadata for node in all_nodes_embedded}

# Initialize the SimpleVectorStore with the dictionaries
vector_store = SimpleVectorStore(
    data = SimpleVectorStoreData(
        embedding_dict=embedding_dict,
        text_id_to_ref_doc_id=text_id_to_ref_doc_id,
        metadata_dict=metadata_dict,
    ),
    stores_text=True
)

## NebulaPropertyGraphStore:

In [5]:
graph_store = NebulaPropertyGraphStore(
    space = "PrimeKG",
    username = "root",
    password = "nebula",
    url = "nebula://localhost:9669",
    props_schema= """`node_index` STRING, `node_type` STRING, `node_id` STRING, `node_name` STRING, 
        `node_source` STRING, `mondo_id` STRING, `mondo_name` STRING, `group_id_bert` STRING, 
        `group_name_bert` STRING, `orphanet_prevalence` STRING, `display_relation` STRING """,
)

# LLM

## SciBERT

In [6]:
# tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_cased", padding_side="left", device_map="auto")    
# if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
#     tokenizer.pad_token = "<|end_of_text|>"
#     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# if tokenizer.eos_token_id is None: # no <eos> token previously defined
#     tokenizer.eos_token = "<|end_of_text|>"
#     tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)


# llm = HuggingFaceLLM(
#     model_name="allenai/scibert_scivocab_cased",
#     context_window=512,
#     max_new_tokens=1048,
#     generate_kwargs={
#         "temperature": 0.10, 
#         "do_sample": True,
#         "pad_token_id": tokenizer.pad_token_id,
#         "top_k": 5, 
#         "top_p": 0.85
#     },
#     model_kwargs={
#         "torch_dtype": torch.float16,
#     },
#     tokenizer=tokenizer,
#     # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
#     device_map="cuda" if torch.cuda.is_available() else "cpu",
#     stopping_ids=[tokenizer.eos_token_id],
#     tokenizer_kwargs={"max_length": None},
#     is_chat_model=True,
# )

# Settings.llm = llm
# Settings.chunk_size = 1024
# Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") # BAAI/bge-small-en-v1.5 /  m3

## Llama-3.2-3B-Instruct

In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct", padding_side="left", device_map="auto")    
if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
    tokenizer.pad_token = "<|end_of_text|>"
    tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)


llm = HuggingFaceLLM(
    model_name="meta-llama/Llama-3.2-3B-Instruct",
    context_window=8192,
    max_new_tokens=3048,
    generate_kwargs={
        "temperature": 0.10, 
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "top_k": 10, 
        "top_p": 0.9,
        # "repetition_penalty": 0.9,  # Added to reduce repetition
        # "no_repeat_ngram_size": 3,  # Prevents repetition of n-grams
    },
    model_kwargs={
        "torch_dtype": torch.float16,
    },
    tokenizer=tokenizer,
    # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    stopping_ids=[tokenizer.eos_token_id],
    tokenizer_kwargs={"max_length": None},
    is_chat_model=True,
)

Settings.llm = llm
Settings.chunk_size = 1024
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2") # BAAI/bge-small-en-v1.5 /  m3 / sentence-transformers/all-mpnet-base-v2

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Llama-3.3-70B-Instruct

In [8]:
# # for Llama-3.3-70B-Instruct
# from transformers import BitsAndBytesConfig
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4"
# )

# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct", padding_side="left", device_map="auto")    
# if tokenizer.pad_token_id is None: #no <pad> token previously defined, only eos_token
#     tokenizer.pad_token = "<|end_of_text|>"
#     tokenizer.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)


# llm = HuggingFaceLLM(
#     model_name="meta-llama/Llama-3.3-70B-Instruct",
#     context_window=1048,
#     max_new_tokens=256,
#     generate_kwargs={
#         "temperature": 0.10, 
#         "do_sample": True,
#         "pad_token_id": tokenizer.pad_token_id,
#         "top_k": 5, 
#         "top_p": 0.85
#     },
#     model_kwargs={
#         "torch_dtype": torch.float16,
#         "quantization_config": quantization_config,
#     },
#     tokenizer=tokenizer,
#     # device_map="auto",  # Automatically offload layers to CPU if GPU memory is insufficient
#     device_map="cuda" if torch.cuda.is_available() else "cpu",
#     stopping_ids=[tokenizer.eos_token_id],
#     tokenizer_kwargs={"max_length": None},
#     is_chat_model=True,
# )

# Settings.llm = llm
# Settings.chunk_size = 1024
# Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5") # BAAI/bge-small-en-v1.5 /  m3

# DiseaseMode

MetadataFilters:

In [67]:
from llama_index.core.vector_stores.types import MetadataFilters, FilterOperator
disease_dict = {
    "key": "node_type",
    "value": "disease",
    "operator": FilterOperator.EQ
}
disease_filter = MetadataFilters(filters=[disease_dict])

class DiseaseMode():
    def __init__(self, vector_store: SimpleVectorStore, graph_store: NebulaPropertyGraphStore):
        self.vector_store = vector_store
        self.graph_store = graph_store

    def retrieve(self, query: str):        
        query_embedding = Settings.embed_model.get_text_embedding(query)
        vector_results = self.vector_store.query(
            VectorStoreQuery(
                query_embedding=query_embedding, 
                similarity_top_k=1,
                filters=disease_filter,
            )
        )
        
        top_node_id = vector_results.ids[0]
        top_node_score = vector_results.similarities[0]
        kg_node = self.graph_store.get(ids=[top_node_id])[0]
                
        results = [{ # Create results list with primary node
            "node_index": kg_node.id_,
            "node_name": kg_node.properties["node_name"],
            "text": getattr(kg_node, "text", ""),
            "score": top_node_score
        }]
        
        print(f"Best node from vector query: Node ID: {kg_node.id_}, "
              f"Score: {top_node_score:.4f}, Name: {kg_node.properties['node_name']}")
        
        # Find related nodes through graph query
        graph_nodes = self.graph_store.structured_query(
            """
            MATCH (e:Node__) WHERE id(e) == $ids
            MATCH p=(e)-[r:Relation__{label:"disease-disease"}]-(t) 
            UNWIND relationships(p) as rel
            RETURN DISTINCT id(t), t.Props__.node_name, t.Chunk__.text
            """, 
            param_map={"ids": top_node_id}
        )
        
        # Calculate similarity for related nodes and add relevant ones to results
        all_similarities = []
        for node in graph_nodes:
            node_text = node["t.Props__.node_name"] + ": " + node["t.Chunk__.text"]
            node_embedding = Settings.embed_model.get_text_embedding(node_text)
                    
            similarity = dot(query_embedding, node_embedding) / (norm(query_embedding) * norm(node_embedding))
            all_similarities.append((node, similarity))
            
        if len(all_similarities) > 3:
            sim = [s for _, s in all_similarities]
            threshold = np.percentile(sim, 75) # keep top 25% of nodes
        else:
            threshold = 0.7
        
        for node, similarity in all_similarities:
            if similarity > threshold: #and similarity >= 0.5:
                results.append({
                    "node_index": node["id(t)"],
                    "node_name": node["t.Props__.node_name"],
                    "text": node_text,
                    "score": similarity
                })
        
        results = sorted(results, key=lambda x: x["score"], reverse=True)
        print("\nBest related nodes from graph query:")
        for node in results:  # Skip primary node
            print(f"ID: {node['node_index']} | node name: {node['node_name']} | score: {node['score']:.4f}")
        
        graph_phenotype = graph_store.structured_query(
            """
            MATCH (e:Node__) WHERE id(e) == $ids
            MATCH (e)-[r:Relation__{label:"disease-phenotype-positive"}]-(t) 
            RETURN DISTINCT id(t), t.Props__.node_name
            """, 
            param_map={"ids": top_node_id}
        )
        # join the phenotype names without the " '' " characters
        phenotypes = ", ".join(node["t.Props__.node_name"].replace("'", "") for node in graph_phenotype)

        nodes_with_text = [node for node in results if node['text'].strip()]
        context = [f"'{node['node_name']}': {node['text']}" for node in nodes_with_text] if nodes_with_text else None
        phenotype_context = [f"Is associated with the following phenotypes: {phenotypes}\n"] if phenotypes else None
        
        if results:
            return (context, phenotype_context, top_node_id)
        else:
            return [f"No graph relationships found for {results[0]['node_name']}"] if results else ["No results found"]


## Prompt templates:

In [ ]:
context_phenotype_template = """    
    Context information is below:
    {text_chunks}

    Phenotype context is below:
    {phenotype_context}

    You are a medical knowledge assistant specializing in rare diseases. Your task is to create a comprehensive list of symptoms for {query_str}.

    CRITICAL INSTRUCTIONS:
    1. Use the information from the context and your own knowledge to provide a comprehensive answer
    2. Return MAXIMUM the 16 most relevant symptoms, if there are more than 16 symptoms, return the most relevant ones
    3. Use HPO medical terminology and avoid using including redundant symptoms
    4. Return EXACTLY this JSON format (no variations):

    Always format your response as a VALID JSON:
    {
        "disease": "{query_str}",
        "symptoms": [
            "name symptom1 using HPO terminology",
            "name symptom2 using HPO terminology",
            ... and so on
        ]
    }

    Do NOT use nested objects. Use exactly "disease" and "symptoms" as shown.
"""

no_rag_template = """
    You are a medical knowledge assistant specializing in rare diseases. Your task is to create a comprehensive list of symptoms for {query_str}.

    CRITICAL INSTRUCTIONS:
    1. Use only your own knowledge to provide a comprehensive answer
    2. Return MAXIMUM the 16 most relevant symptoms, if there are more than 16 symptoms, return the most relevant ones
    3. Use only HPO medical terminology and avoid including redundant symptoms
    4. Return EXACTLY this JSON format (no variations):

    Always format your response as a VALID JSON:
    {
        "disease": "{query_str}",
        "symptoms": [
            "name symptom1 using HPO terminology",
            "name symptom2 using HPO terminology",
            "name symptom3 using HPO terminology",
            ...
        ]
    }

    Do NOT use nested objects. Use exactly "disease" and "symptoms" as shown.
"""


## Chat

In [ ]:
user = "Developmental and epileptic encephalopathy 4" # "autosomal recessive polycystic kidney disease"
print(f"\n=== RAG === \nUser query: {user}\n")
context, phenotypes, top_node_id = DiseaseMode(vector_store, graph_store).retrieve(user)

class Output(BaseModel):
    disease: str = Field(description="The disease of interest given by the user.")
    symptoms: List[str] = Field(description="Symptoms associated with the disease.")
output_parser = PydanticOutputParser(Output)

def get_prompt_and_inputs(context, phenotypes):
    prompt_template = PromptTemplate(context_phenotype_template)
    if context and phenotypes:
        return prompt_template, "\n".join(context), phenotypes
    elif context and not phenotypes:
        return prompt_template, "\n".join(context), ""
    elif not context and phenotypes:
        return prompt_template, "", phenotypes
    else:
        return None, None, None

prompt_template, text_chunks, phenotype_context = get_prompt_and_inputs(context, phenotypes)

if prompt_template:
    print(f"Context: {str(context)[:100]}" if context is not None else context)
    print(f"Phenotypes: {phenotypes[:100] if phenotypes is not None else phenotypes}\n")
    summarizer = TreeSummarize(verbose=True, llm=llm, summary_template=prompt_template)
    response = summarizer.get_response(
        query_str=user,
        text_chunks=text_chunks,
        phenotype_context=phenotype_context
    )
else:
    print(f"No context or phenotypes found for {user}")
    response = "No context or phenotypes found for the given disease."

try:
    rag_response = output_parser.parse(response)
    print("\nRESPONSE OK:", rag_response.model_dump_json())
    print(f"Symptoms retrieved: {len(rag_response.symptoms)} for {rag_response.disease}")
except ValueError as e:
    print(f"Warning ERROR: {e}")
    # Fallback: create an Output object with empty symptoms and the disease name
    rag_response = Output(disease=user, symptoms=[])
    print("\n", rag_response.model_dump_json())
    print(f"{len(rag_response.symptoms)} symptoms for {rag_response.disease}")


print(f"\n\n === no RAG === \nUser query: {user}\n")
# include a prompt template for the LLM chat 
template = PromptTemplate(no_rag_template)
prompt = template.format(query_str=user)
response = llm.chat([ChatMessage(role="user", content=prompt)])

# Extract the text content from the ChatResponse object
response_text = response.message.content if hasattr(response, "message") else str(response)

try:
    no_rag_response = output_parser.parse(response_text)
    print("\nRESPONSE OK:", no_rag_response.model_dump_json())
    print(f"Symptoms retrieved: {len(no_rag_response.symptoms)} for {no_rag_response.disease}")
except ValueError as e:
    print(f"Warning ERROR: {e}")
    # Fallback: create an Output object with empty symptoms and the disease name
    no_rag_response = Output(disease=user, symptoms=[])
    print("\n", no_rag_response.model_dump_json())
    print(f"{len(no_rag_response.symptoms)} symptoms for {no_rag_response.disease}")


=== RAG === 
User query: Developmental and epileptic encephalopathy 4

Best node from vector query: Node ID: 28987, Score: 0.7868, Name: developmental and epileptic encephalopathy, 85, with or without midline brain defects

Best related nodes from graph query:
ID: 28987 | node name: developmental and epileptic encephalopathy, 85, with or without midline brain defects | score: 0.7868
ID: 28005 | node name: developmental and epileptic encephalopathy | score: 0.7537
Context: ["'developmental and epileptic encephalopathy': developmental and epileptic encephalopathy: Any earl
Phenotypes: ['Is associated with the following phenotypes: Cleft palate, Retrognathia, Posteriorly rotated ears, Anteverted nares, Autistic behavior, Growth delay, Hypertonia, Delayed speech and language development, Stereotypy, Atrial septal defect, Talipes, Camptodactyly, Clinodactyly, Seizure, Hirsutism, Focal-onset seizure, Generalized hypotonia, Overfolded helix, Facial asymmetry, Choanal atresia, Widely spaced t

In [14]:
# load the disease_to_symptoms dictionary from a json file
output_file = os.path.expanduser('~/scratch-llm/storage/phenopackets/phenopacket_data.json')
with open(output_file, 'r') as f:
    phenopackets = json.load(f)

In [52]:
# compare the the symptoms from the RAG and no-RAG responses

common_symptoms = set(rag_response.symptoms).intersection(set(no_rag_response.symptoms))
print(f"\nCommon symptoms: {common_symptoms}")


# compare the two responses with the phenopackets dictionary
# Find the entry in phenopackets where the disease name matches the user
phenopacket_diseases = phenopackets[user]
# Convert all symptoms to lowercase for case-insensitive comparison

print(f"Common symptoms between RAG and phenopackets: {set(s.lower() for s in rag_response.symptoms).intersection(set(d.lower() for d in phenopacket_diseases))}")
print(f"Common symptoms between no-RAG and phenopackets: {set(s.lower() for s in no_rag_response.symptoms).intersection(set(d.lower() for d in phenopacket_diseases))}")


Common symptoms: set()
Common symptoms between RAG and phenopackets: {'delayed speech and language development', 'eeg abnormality', 'seizure', 'hypotonia', 'bilateral tonic-clonic seizure', 'severe global developmental delay', 'global developmental delay', 'multifocal seizures', 'infantile spasms', 'hypsarrhythmia', 'absent speech', 'focal-onset seizure'}
Common symptoms between no-RAG and phenopackets: {'hypotonia', 'microcephaly', 'apnea', 'cerebral atrophy', 'feeding difficulties', 'autism', 'encephalopathy'}


In [55]:
# remove the terms in rag_response.symptoms and no_rag_response.symptoms that are in the phenopackets dictionary
rag_response.symptoms = [s for s in rag_response.symptoms if s.lower() not in {d.lower() for d in phenopacket_diseases}]
no_rag_response.symptoms = [s for s in no_rag_response.symptoms if s.lower() not in {d.lower() for d in phenopacket_diseases}]

In [56]:
phenotype_dict = {
    "key": "node_type",
    "value": "effect/phenotype",
    "operator": FilterOperator.EQ
}
phenotype_filter = MetadataFilters(filters=[phenotype_dict])


def find_HPO_embedding(symptoms: List[str]) -> List[str]:
    """Get the embedding for each phenotype term that has not been previously matched and map it to the the most similar HPO term."""

    new_symptoms = []
    for term in symptoms.symptoms:
        query_embedding = Settings.embed_model.get_text_embedding(term)
        vector_results = vector_store.query(
            VectorStoreQuery(
                    query_embedding=query_embedding, 
                    similarity_top_k=1,
                    filters=phenotype_filter,
                )
            )
        kg_node = graph_store.get(ids=[vector_results.ids[0]])

        # indicate if querying rag or no-rag
        print(f"Term: {term} | Vector ID: {vector_results.ids[0]} | KG node: {kg_node[0].properties['node_name']} | Similarity: {vector_results.similarities[0]:.4f}")
        
        # replace the term in no_rag_response.symptoms with the KG node nam
        if vector_results.similarities[0] > 0.5 and kg_node:
            new_symptoms.append(kg_node[0].properties['node_name'])
        else:
            new_symptoms.append(term)
            
    return list(set(new_symptoms))

no_rag_response.symptoms = find_HPO_embedding(no_rag_response)
rag_response.symptoms = find_HPO_embedding(rag_response)

common_symptoms = set(rag_response.symptoms).intersection(set(no_rag_response.symptoms))
print(f"\nCommon symptoms (after clean-up): {common_symptoms}")

# compare the two responses again with the phenopackets dictionary
print(f"\nCommon symptoms between RAG and phenopackets (after clean up): {set(s.lower() for s in rag_response.symptoms).intersection(set(d.lower() for d in phenopacket_diseases))}")
print(f"Common symptoms between no-RAG and phenopackets: {set(s.lower() for s in no_rag_response.symptoms).intersection(set(d.lower() for d in phenopacket_diseases))}")


Term: Hypomanganesemia | Vector ID: 91569 | KG node: Hypomanganesemia | Similarity: 0.8117
Term: Encephalomalacia | Vector ID: 92874 | KG node: Encephalomalacia | Similarity: 0.8392
Term: Sensory neuropathy | Vector ID: 23456 | KG node: Sensory neuropathy | Similarity: 0.8459
Term: Peripheral hypomyelination | Vector ID: 86942 | KG node: Peripheral hypomyelination | Similarity: 0.9417
Term: Cerebral edema | Vector ID: 84424 | KG node: Cerebral edema | Similarity: 0.8580
Term: Microlissencephaly | Vector ID: 94301 | KG node: Microlissencephaly | Similarity: 0.8602
Term: Poor eye contact | Vector ID: 84722 | KG node: Poor eye contact | Similarity: 0.7474
Term: Synophrys | Vector ID: 84681 | KG node: Synophrys | Similarity: 0.6477
Term: Muscular hypotonia of the trunk | Vector ID: 24642 | KG node: Muscular hypotonia of the trunk | Similarity: 0.9504
Term: Hypotelorism | Vector ID: 84663 | KG node: Hypotelorism | Similarity: 0.7622
Term: Seizure cluster | Vector ID: 92487 | KG node: Seizur

In [ ]:
from pyhpo import Ontology
_ = Ontology()
# Compare the parent terms of rag_response.symptoms to the ones in phenopacket_diseases
for term in rag_response.symptoms:
    hpo_term = Ontology.get_hpo_object(term)
    if hpo_term:
        if hpo_term.parents:
            for parent in hpo_term.parents:
                parent_name = parent.name.lower()
                if parent_name in {d.lower() for d in phenopacket_diseases}:
                    print(f"Match found {parent.id} - {parent.name}")
                else:
                    print(f"NO match for {parent.id} - {parent.name}")
        else:
            print(f"Term: {hpo_term.name} | Parent: None")
    else:
        print(f"Term {term} not found in HPO ontology.")

NO match for HP:0032096 - Abnormal manganese concentration
Match found HP:0009830 - Peripheral neuropathy
NO match for HP:0003130 - Abnormal peripheral myelination
NO match for HP:0000969 - Edema
NO match for HP:0002060 - Abnormal cerebral morphology
NO match for HP:0001339 - Lissencephaly
NO match for HP:0002060 - Abnormal cerebral morphology


#### Checks with NebulaGraph 

In [63]:
%ngql CREATE SPACE IF NOT EXISTS PrimeKG(vid_type=FIXED_STRING(256))
%ngql USE PrimeKG;

""


In [ ]:
%%ngql
MATCH (e:Node__)
WHERE id(e) == "38849"
RETURN DISTINCT 
        e.Props__.node_name, 
        e.Chunk__.text

,e.Props__.node_name,e.Chunk__.text
0,Wieacker-Wolff syndrome (spectrum),


In [66]:
%%ngql
# find all disease nodes where e.Chunk__.text is empty
MATCH (e:Node__)
MATCH (e)-[r1:Relation__{label:"disease-phenotype-positive"}]-(t1) # {label:"disease-phenotype-positive"}
WHERE e.Props__.node_type == "disease" AND e.Chunk__.text == ""
RETURN DISTINCT 
    e.Props__.node_name,
    e.Props__.node_type,
    e.Chunk__.text,
    t1.Props__.node_name,
    t1.Props__.node_type

,e.Props__.node_name,e.Props__.node_type,e.Chunk__.text,t1.Props__.node_name,t1.Props__.node_type
0,familial focal alopecia,disease,,Autosomal dominant inheritance,effect/phenotype
1,familial focal alopecia,disease,,Patchy alopecia,effect/phenotype
2,myelolymphatic insufficiency,disease,,Hyposegmentation of neutrophil nuclei,effect/phenotype
3,myelolymphatic insufficiency,disease,,X-linked recessive inheritance,effect/phenotype
4,"cerebellar, ocular, craniofacial, and genital syndrome",disease,,Low-set ears,effect/phenotype
...,...,...,...,...,...
12449,"Wieacker-Wolff syndrome, female-restricted",disease,,Inability to walk,effect/phenotype
12450,"Wieacker-Wolff syndrome, female-restricted",disease,,Distal muscle weakness,effect/phenotype
12451,"Wieacker-Wolff syndrome, female-restricted",disease,,Kyphosis,effect/phenotype
12452,"Wieacker-Wolff syndrome, female-restricted",disease,,Flexion contracture,effect/phenotype


In [ ]:
# %%ngql
# MATCH (e:Node__)
# WHERE id(e) == "27781"
# MATCH (e)-[r1:Relation__{label:"disease-phenotype-positive"}]-(t1) # {label:"disease-phenotype-positive"}
# # MATCH (e)-[r2:Relation__{label:"disease-disease"}]-(t2)
# # RETURN e.Props__.node_name, e.Chunk__.text

# RETURN DISTINCT 
#         e.Props__.node_name, 
#         # e.Chunk__.text
#         r1.label,
#         id(t1),
#         t1.Props__.node_name
#         # r2.label,
#         # id(t2) AS index_disease,
#         # t2.Props__.node_name AS name_disease
#         # t.Chunk__.text


In [69]:
n.stop()

udocker command: udocker /home/lasa14/.conda/envs/llm-rag/bin/udocker --allow-root ps | grep graphd | awk '{print $1}' | xargs -I {} udocker --allow-root rm -f {}
error: Error: deleting directory:  /home/lasa14/.udocker/containers/d9a1b12a-62b5-3764-80e3-dd482130cc1d
Error: deleting container

Retrying in 5 seconds... udocker command failed with return code 123
udocker command: udocker /home/lasa14/.conda/envs/llm-rag/bin/udocker --allow-root ps | grep graphd | awk '{print $1}' | xargs -I {} udocker --allow-root rm -f {}
error: Error: deleting directory:  /home/lasa14/.udocker/containers/d9a1b12a-62b5-3764-80e3-dd482130cc1d
Error: deleting container

Retrying in 15 seconds... udocker command failed with return code 123
udocker command: udocker /home/lasa14/.conda/envs/llm-rag/bin/udocker --allow-root ps | grep graphd | awk '{print $1}' | xargs -I {} udocker --allow-root rm -f {}
error: Error: deleting directory:  /home/lasa14/.udocker/containers/d9a1b12a-62b5-3764-80e3-dd482130cc1d
Err